In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime

# Step 1: Load Data
file_path = r"C:\Users\rithi\OneDrive\Documents\visual studio\rithika - traffic_stops.csv"
df = pd.read_csv(file_path)


In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="yourpassword",
    database="yourdb"
)
cursor = conn.cursor()

In [ ]:
# Step 2: Data Cleaning

# Drop columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Fill NaNs with suitable values
df.fillna({
    'driver_age': df['driver_age'].median(),
    'search_type': 'None',
    'stop_duration': 'Unknown',
    'violation': 'Unknown',
    'stop_outcome': 'Unknown',
}, inplace=True)

# Convert date & time to timestamp
#df['timestamp'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])


In [ ]:
import pymysql

# Connect to MySQL
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="1234",
)

cursor = conn.cursor()
cursor.execute("create database securecheck")



In [ ]:
cursor.execute("USE securecheck;")

cursor.execute("""
CREATE TABLE IF NOT EXISTS police_logs (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stop_date DATE,
    stop_time TIME,
    county_name VARCHAR(100),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    drugs_related_stop BOOLEAN,
    stop_duration VARCHAR(50),
    vehicle_number VARCHAR(50),
    timestamp DATETIME
)
""")


In [ ]:
# ✅ Iterate over DataFrame rows and insert
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO police_logs (
            stop_date, stop_time, county_name, driver_gender, driver_age, driver_race,
            violation, search_conducted, search_type, stop_outcome, drugs_related_stop,
            stop_duration, vehicle_number, timestamp
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['stop_date'],
        row['stop_time'],
        row['county_name'],
        row['driver_gender'],
        int(row['driver_age']),
        row['driver_race'],
        row['violation'],
        bool(row['search_conducted']),
        row['search_type'],
        row['stop_outcome'],
        bool(row['drugs_related_stop']),
        row['stop_duration'],
        row.get('vehicle_number', None),  # safely get vehicle_number if it exists
        row['timestamp']
    ))

# ✅ Commit all inserts and close
conn.commit()
cursor.close()
conn.close()

print("✅ All rows successfully inserted into police_logs table!")